# Analysis IFG Script

Analyzes a functional group dataset and a bandgap dataset for quantitative structure-property relationships. 

Before running this script, ensure that IFG had been installed to your computer.  <br>
To see more information about this script, please view the doc page for this script at the [IFG docs](https://wtriddle.github.io/IFG/).

## Script Modules and Packages

Imported modules and usages:

- <b>itertools</b> implemenets a powerset in python
- <b>math</b> executes statistically relevant operations in python
- <b>os</b> gets file paths for python to locate files on computer file system
- <b>numpy</b> finds averages in python with speed
- <b>pandas</b> tabulates functional group python data and exports it to excel 

In [5]:
from itertools import chain, combinations
import math
import os

import numpy
import pandas

## Script I/O Files

File path to functional groups excel file

In [6]:
MAIN_OUTPUT_PATH = os.getcwd() + '/output/functional_groups.xlsx'
MAIN_OUTPUT_PATH

'c:\\Users\\wtrid\\Documents\\Software Development\\IFG\\ifg\\scripts/output/functional_groups.xlsx'

File path to excel file that will be generated by this script

In [7]:
ANALYSIS_OUTPUT_PATH = os.getcwd() + '/output/stats.xlsx'
ANALYSIS_OUTPUT_PATH

'c:\\Users\\wtrid\\Documents\\Software Development\\IFG\\ifg\\scripts/output/stats.xlsx'

File path to bandgaps excel file

In [8]:
BANDGAPS_PATH = os.getcwd() + '/output/CrystalData.xlsx'
BANDGAPS_PATH

'c:\\Users\\wtrid\\Documents\\Software Development\\IFG\\ifg\\scripts/output/CrystalData.xlsx'

## Script Excel Interface

The object below is the <b>xlsxwriter</b> python interface for python data to transfer into the excel filed targeted by the <b>ANALYSIS_OUTPUT_PATH</b> file. Please set this up

In [9]:
writer = pandas.ExcelWriter(ANALYSIS_OUTPUT_PATH)

## Script Data Set-Up

Excel file data load into tabulated <b>pandas</b> dataframes

The code below extracts the functional groups and bandgaps datasets from the input excel files into <b>pandas</b> dataframes. The column of bandgaps are concatinated to the functional groups dataframe into a new dataframe called <b>bandgap_fg_data</b>, where each row is a molecule with its functional groups and bandgap property data. Run the code below to load this data. 

Please note that the data loading logic can change based on your data format and your needs for using this script.

In [10]:
bandgap_data: pandas.Series = pandas.read_excel(BANDGAPS_PATH, sheet_name="All")["bandgap"]
fg_data: pandas.DataFrame = pandas.read_excel(MAIN_OUTPUT_PATH, sheet_name="exact_data").drop("AminoAcid", axis=1)
bandgap_fg_data: pandas.DataFrame = pandas.concat([fg_data, bandgap_data], axis=1).set_index("Refcode").drop("SMILES", axis=1)
bandgap_fg_data

,Aromatic Rings,Non Aromatic Rings,Rings,Amide,Alkyne,Non Aromatic Amide,Non Aromatic TertiaryAmine,Nitro,Ether,CarboxylicAcid,...,HydroPeroxide,Non Aromatic Carbonate,Non Aromatic Alkyne,Isonitrile,Aromatic TertiaryAmine,Aldoxime,Non Aromatic Peroxide,Carbonate,Acetal,bandgap
Refcode,,,,,,,,,,,,,,,,,,,,,
ABAWEG01,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.882338
ABEJIC,5,0,5,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.350893
ABEJOH,0,2,2,0,0,2,2,3,0,0,...,0,0,0,0,0,0,0,0,0,3.715734
ABEWAG,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,5.041824
ACETAC07,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,5.218628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YURRIN,2,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.154905
ZILFUV,1,1,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.808635
ZOJRAR01,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.395971


Bandgap row category set-up

The statistical matrix formats computed in this script (shown visually on the script [docs page](https://wtriddle.github.io/IFG/scripts/data_analysis.html#staistical-matrix-script-output)) uses numerical ranges of bandgap value as its row categproes. These bandgap categories called <b>bandgap_bins</b> are created in 0.5eV numerical range denominations starting from 0eV up to the maximum bandgap value in the <b>bandgap_fg_data</b> dataset called <b>max_bandgap</b>. Note that the value shown below in the list is the lower limit of each range. As an example, entry 0 is a range of 0eV-0.5eV.

In [11]:
max_bandgap: int = max(bandgap_fg_data["bandgap"])
bandgap_bins: "list[float]" = [x*0.5 for x in range(0, 2*math.ceil(max_bandgap))]
print(bandgap_bins)

[0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5]


Molecule categorizations by numerical bandgap ranges

The <b>bandgap_bins</b> numerical bandgap range categories are applied to the <b>bandgap_fg_data</b> to seperate molecules out into bandgap range dependent sub-tables. Below, this is achieved by extracting all rows in the <b>bandgap_fg_data</b> dataframe which fall into a particular bandgap range into their own dataframe, <b>bandgap_range_sorted_molecule_dataframes</b>. 

In [12]:
bandgap_range_sorted_molecule_dataframes: list[pandas.DataFrame] = [
    bandgap_fg_data.loc[
        (bandgap_fg_data['bandgap'] >= bandgap) & 
        (bandgap_fg_data['bandgap'] < bandgap+0.5)
    ]
    for bandgap in bandgap_bins
]

Example of a bandgap numerical range sub-table

In [13]:
print(f'range: {bandgap_bins[3]} eV - {bandgap_bins[3] + 0.5} eV')
bandgap_range_sorted_molecule_dataframes[3]

range: 1.5 eV - 2.0 eV


,Aromatic Rings,Non Aromatic Rings,Rings,Amide,Alkyne,Non Aromatic Amide,Non Aromatic TertiaryAmine,Nitro,Ether,CarboxylicAcid,...,HydroPeroxide,Non Aromatic Carbonate,Non Aromatic Alkyne,Isonitrile,Aromatic TertiaryAmine,Aldoxime,Non Aromatic Peroxide,Carbonate,Acetal,bandgap
Refcode,,,,,,,,,,,,,,,,,,,,,
ATUNOT,3,1,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.733953
ATUPAH,3,1,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.680069
AVOJUR,1,0,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1.736792
AXEGAM,1,3,4,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,1.964960
AYUVOG,2,1,3,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1.863213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YAGWEK,2,1,3,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1.680224
YIVQEA,3,0,3,0,0,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,1.701348
YODKAE,1,1,2,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1.564355


Another example of a bandgap numerical range sub-table

In [14]:
print(f'range: {bandgap_bins[4]} eV - {bandgap_bins[4] + 0.5} eV')
bandgap_range_sorted_molecule_dataframes[4]

range: 2.0 eV - 2.5 eV


,Aromatic Rings,Non Aromatic Rings,Rings,Amide,Alkyne,Non Aromatic Amide,Non Aromatic TertiaryAmine,Nitro,Ether,CarboxylicAcid,...,HydroPeroxide,Non Aromatic Carbonate,Non Aromatic Alkyne,Isonitrile,Aromatic TertiaryAmine,Aldoxime,Non Aromatic Peroxide,Carbonate,Acetal,bandgap
Refcode,,,,,,,,,,,,,,,,,,,,,
ABEJIC,5,0,5,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.350893
ADUWUS01,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.390699
ANOBAH,3,1,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.164785
AYUGEH,0,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2.213924
AZAMOE,2,2,4,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.323235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YIRMAO,6,2,8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.388793
YODFUT,1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2.358908
YOTSUW,1,0,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2.034325


Molecule validity count

To ensure that the bandgap numerical ranges did not erroneously miss any molecule entries in the generated sub-tables, please run the code cell below

In [31]:
molecule_set_sizes: list[int] = [len(bandgap_molecules_subtable) for bandgap_molecules_subtable in bandgap_range_sorted_molecule_dataframes]
assert sum(molecule_set_sizes) == len(bandgap_fg_data)

## Statistical Metrics Computation Method

The matrix should include a mean and standard deviation for each functional group column in terms of bandgap in eV. The method below takes in a matrix and computes this for each of its columns. Please set up this method below.

In [16]:
def createStatisticalMetrics(matrix: pandas.DataFrame, bandgap_bins: "list[float]"):
    """Create the bandgap average and bandgap standard deviation for every functional group column (category) in the given matrix"""

    averages: list[float] = [
        float(numpy.average(a=bandgap_bins, weights=matrix[fg])) 
        for fg in matrix.columns
    ]

    variances: list[float] = [
        float(numpy.average(a=[(bandgap-averages[i])**2 for bandgap in bandgap_bins], weights=matrix[fg])) 
        for i, fg in enumerate(matrix.columns)
    ]

    return (averages, [math.sqrt(variance) for variance in variances])


## Independent Matrix

### Independent Matrix Details

The independent matrix is a matrix with <b>bandgap_bins</b> rows and singular functional group columns. Its cell values are the number of molecules in the dataset which have a particular functional group and which exhibit a value of bandgap in a particular numerical bandgap range. The columns are independent of one another, while the numerical bandgap ranges are depenednet on each other. This means that the count of a structure can only fall into one row per column (since a structure can only fall into one bandgap numerical range), but the count of a structure can appear across any other columns multiple times (since a structure can have multiple different functional groups). The columns of the independent matrix statistically isolate functional groups from each other, and only considers the behavior of a single functional group per column.

### Independent Matrix Computation
The independent matrix data is constructed below with an <b>independent_data</b> python list comprehension. The list comprehension iterates over each bandgap-dependent sub-table (dataframe) <b>bandgap_molecules_subtable</b> in <b>bandgap_range_sorted_molecule_dataframes</b> and determines the number of rows in each sub-table which show the presence of a particular functional group in any amount of apperance frequency (i.e. at least one apperance). Applying all functional groups to each bandgap-dependent sub-table yields all of the entries in the independent matrix. 

Once the counting is completed by the <b>independent_data</b>, <b>pandas</b> converts the <b>independent_data</b> into an <b>independent_matrix</b> with singular functional group columns and bandgap numerical range rows. The resultant <b>independent_matrix</b> cell value is the number of molecules in a particular bandgap numerical range and which had a particular functional group. The total number of molecules in each bandgap numerical range is computed as well.

In [17]:
independent_data: list[list[int]] = [
    [len(bandgap_molecules_subtable.loc[bandgap_molecules_subtable[f] > 0]) for f in bandgap_fg_data.columns]
    for bandgap_molecules_subtable in bandgap_range_sorted_molecule_dataframes
]
independent_matrix: pandas.DataFrame = pandas.DataFrame(data=independent_data, columns=bandgap_fg_data.columns, index=bandgap_bins)
independent_matrix["total"] = [sum(d) for d in independent_data]
independent_matrix

,Aromatic Rings,Non Aromatic Rings,Rings,Amide,Alkyne,Non Aromatic Amide,Non Aromatic TertiaryAmine,Nitro,Ether,CarboxylicAcid,...,Non Aromatic Carbonate,Non Aromatic Alkyne,Isonitrile,Aromatic TertiaryAmine,Aldoxime,Non Aromatic Peroxide,Carbonate,Acetal,bandgap,total
0.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0.5,4,2,4,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,4,20
1.0,21,12,23,0,1,5,2,8,2,1,...,0,0,0,1,0,0,0,0,23,140
1.5,58,38,63,2,3,3,6,24,15,2,...,0,0,0,1,0,0,0,0,63,417
2.0,98,65,115,7,4,13,15,31,33,10,...,0,0,0,1,0,0,0,0,117,748
2.5,143,109,169,17,3,16,23,18,28,11,...,0,0,0,1,1,0,1,1,171,1088
3.0,103,107,142,19,3,13,25,6,23,17,...,0,1,1,0,0,1,1,0,154,936
3.5,71,94,112,16,3,23,25,2,17,8,...,0,1,0,0,1,1,0,0,119,717
4.0,33,56,65,10,0,10,12,1,6,5,...,1,0,0,0,0,1,0,0,72,406
4.5,1,37,38,5,0,10,9,0,3,2,...,1,0,0,0,0,0,0,0,47,231


Independent Matrix Statistical Metrics

The code below computes the bandgap mean and standard deviation of each column of the <b>independent_matrix</b> in eV, as well as the total number of molecules that appear in each functional group column. It adds them as three new rows below the <b>independent_matrix</b>. Run the code below to add the rows and view the updated matrix.

In [18]:
independent_means: list[float]
independent_stds: list[float]
independent_means, independent_stds = createStatisticalMetrics(independent_matrix, bandgap_bins)
independent_matrix.loc["total"] = list(independent_matrix.sum())              # type: ignore
independent_matrix.loc["mean"] = list(independent_means)                      # type: ignore
independent_matrix.loc["std"] = list(independent_stds)                        # type: ignore
independent_matrix

,Aromatic Rings,Non Aromatic Rings,Rings,Amide,Alkyne,Non Aromatic Amide,Non Aromatic TertiaryAmine,Nitro,Ether,CarboxylicAcid,...,Non Aromatic Carbonate,Non Aromatic Alkyne,Isonitrile,Aromatic TertiaryAmine,Aldoxime,Non Aromatic Peroxide,Carbonate,Acetal,bandgap,total
0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.000000
0.5,4.000000,2.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.00000,1.000000,0.000000,...,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.00,0.0,4.000000,20.000000
1.0,21.000000,12.000000,23.000000,0.000000,1.000000,5.000000,2.000000,8.00000,2.000000,1.000000,...,0.000000,0.00,0.0,1.000000,0.0,0.000000,0.00,0.0,23.000000,140.000000
1.5,58.000000,38.000000,63.000000,2.000000,3.000000,3.000000,6.000000,24.00000,15.000000,2.000000,...,0.000000,0.00,0.0,1.000000,0.0,0.000000,0.00,0.0,63.000000,417.000000
2.0,98.000000,65.000000,115.000000,7.000000,4.000000,13.000000,15.000000,31.00000,33.000000,10.000000,...,0.000000,0.00,0.0,1.000000,0.0,0.000000,0.00,0.0,117.000000,748.000000
2.5,143.000000,109.000000,169.000000,17.000000,3.000000,16.000000,23.000000,18.00000,28.000000,11.000000,...,0.000000,0.00,0.0,1.000000,1.0,0.000000,1.00,1.0,171.000000,1088.000000
3.0,103.000000,107.000000,142.000000,19.000000,3.000000,13.000000,25.000000,6.00000,23.000000,17.000000,...,0.000000,1.00,1.0,0.000000,0.0,1.000000,1.00,0.0,154.000000,936.000000
3.5,71.000000,94.000000,112.000000,16.000000,3.000000,23.000000,25.000000,2.00000,17.000000,8.000000,...,0.000000,1.00,0.0,0.000000,1.0,1.000000,0.00,0.0,119.000000,717.000000
4.0,33.000000,56.000000,65.000000,10.000000,0.000000,10.000000,12.000000,1.00000,6.000000,5.000000,...,1.000000,0.00,0.0,0.000000,0.0,1.000000,0.00,0.0,72.000000,406.000000
4.5,1.000000,37.000000,38.000000,5.000000,0.000000,10.000000,9.000000,0.00000,3.000000,2.000000,...,1.000000,0.00,0.0,0.000000,0.0,0.000000,0.00,0.0,47.000000,231.000000


### Independent Matrix To Excel Export 

The code below transfers the python pandas dataframe (table) into a sheet of an excel file targeted by the <b>ANALYSIS_OUTPUT_PATH</b> variable. It sets the column widths for each column to allow user visibility of data and column widths. 

In [19]:
independent_matrix.to_excel(writer, sheet_name="independent_matrix", freeze_panes=(1, 1))
independent_sheet = writer.sheets["independent_matrix"]
independent_sheet.set_column(0, 0, 7)
independent_sheet_columns: list[str] = [str(col) for col in independent_matrix.columns][1:]
for i, col in enumerate(independent_sheet_columns):
    independent_sheet.set_column(i+1, i+1, len(col)+7)

## Combinational Matrix

### Combinational Matrix Details

The Combinational Matrix is a statistical functional groups/bandgap matrix which uses functional group sets as its columns and the <b>bandgap_bins</b> as its rows. The column sets are derived from combinations of a root set using a powerset, complement set, and set differences. The result is that the combinational matrix columns and rows are both independent of each other, such that the count of a structure with functional groups and bandgap range can only be conted once per cell. This aspect allows for a percentage stacked bar chart of functional group sets that reveal which sets of functional groups appear commonly in a certain bandgap range.


## Combinational Matrix Set-Up

Functional Group Set Set-Up

Set up the root functional group set to be used for the Combinational Matrix (see the [IFG docs](https://wtriddle.github.io/IFG/scripts/data_analysis.html#combinational-matrix-details) for details on the root functional group set). If you would like to execute with multiple sets in a single runtime, please use the <b>analysis.py</b> script instead.

In [20]:
FG_SET = ["PrimaryAmine", "SecondaryAmine", "TertiaryAmine"]

Method Set-Ups

Set up the two methods below that are relevant for computation of the functional group combinations from the root set

In [21]:
def powerset(iterable):
    """Produce all subsets, including the full set, of a given iterable"""
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))


In [22]:
def generateCombinations(fg_set: "list[str]") -> "list[str]":
    """Generate all combinational functional group queries for a pandas dataframe using the functional group names listed in fg_set"""
    
    return [
        " & ".join(
            [
                "`" + fg + "` > 0" if fg in non_zero_fgs 
                else "`" + fg + "` == 0" 
                for fg in fg_set
            ]
        )
        for non_zero_fgs in list(powerset(fg_set)) if non_zero_fgs
    ]


Stacked Bar Chart Colors Set-Up

Run the code below to set-up the colors that will be used in the stacked percentage chart for the combinational matrix. Please fill the <b>COLORS</b> entries with at least $2^N$ entries, where N is the number of entries in the <b>SET</b> variable.

In [23]:
class Color():
    def __init__(self, label_color: str, bar_color: str):
        self.label_color: str = label_color
        self.bar_color: str = bar_color

COLORS = [
    Color(label_color, bar_color)
    for (label_color, bar_color) in
    [
        ("#ffffff", "#0b84a5"),
        ("#000000", "#f6c85f"),
        ("#ffffff", "#6f4e7c"),
        ("#000000", "#9dd866"),
        ("#ffffff", "#ca472f"),
        ("#000000", "#ffa056"),
        ("#000000", "#8dddd0"),
    ]
]

## Combinational Matrix Computation

Functional Groups Combinational Sets Preparation

Below is the generation of the functional group combinations sets of the combinational matrix columns. To see the set theory derivation of the functional group combinations from the functional groups root set <b>SET</b> and the reasoning behind this type of querying, please look at the [IFG docs](https://wtriddle.github.io/IFG/scripts/data_analysis.html#combinational-matrix-details). The printed output shows the column names and the functional group combinational dataframe queries that represent the combinational functional group sets.

In [24]:
combinational_queries: list[str] = generateCombinations(FG_SET)
combinational_category_names: list[str] = [' '.join(fgs) if len(fgs) > 1 else fgs[0] for fgs in powerset(FG_SET) if fgs]
for (name, query) in zip(combinational_category_names, combinational_queries):
    print(f'{name} | {query}')

PrimaryAmine | `PrimaryAmine` > 0 & `SecondaryAmine` == 0 & `TertiaryAmine` == 0
SecondaryAmine | `PrimaryAmine` == 0 & `SecondaryAmine` > 0 & `TertiaryAmine` == 0
TertiaryAmine | `PrimaryAmine` == 0 & `SecondaryAmine` == 0 & `TertiaryAmine` > 0
PrimaryAmine SecondaryAmine | `PrimaryAmine` > 0 & `SecondaryAmine` > 0 & `TertiaryAmine` == 0
PrimaryAmine TertiaryAmine | `PrimaryAmine` > 0 & `SecondaryAmine` == 0 & `TertiaryAmine` > 0
SecondaryAmine TertiaryAmine | `PrimaryAmine` == 0 & `SecondaryAmine` > 0 & `TertiaryAmine` > 0
PrimaryAmine SecondaryAmine TertiaryAmine | `PrimaryAmine` > 0 & `SecondaryAmine` > 0 & `TertiaryAmine` > 0


Combinational Matrix Data Computation

The code below generates the data for the combinational matrix by iterating over the <b>bandgap_range_sorted_molecule_dataframes</b> with <b>bandgap_molecule_subtable</b> and querying each table for the number of molecules which fall into each functional group combination set. The <b>combinational_data</b> is tabulated into a pandas dataframe (or table), and the total number of molecules in each bandgap row is computed. Any fully zero functional group combinational cateogry (meaning no molecule counts) is eliminated from the dataframe.

In [25]:
combinational_data: list[list[int]] = [
    [len(bandgap_molecule_subtable.query(q)) for q in combinational_queries]
    for bandgap_molecule_subtable in bandgap_range_sorted_molecule_dataframes
]
combinational_matrix: pandas.DataFrame = pandas.DataFrame(data=combinational_data, columns=combinational_category_names, index=bandgap_bins)
combinational_matrix["total"] = [sum(d) for d in combinational_data]
combinational_matrix = combinational_matrix.loc[:, (combinational_matrix != 0).any(axis=0)]
combinational_matrix

,PrimaryAmine,SecondaryAmine,TertiaryAmine,PrimaryAmine SecondaryAmine,total
0.0,0,0,0,0,0
0.5,0,0,0,0,0
1.0,1,2,2,0,5
1.5,4,5,3,1,13
2.0,5,8,2,0,15
2.5,8,6,4,0,18
3.0,11,6,3,3,23
3.5,10,1,1,0,12
4.0,2,3,1,0,6
4.5,2,1,0,0,3


Combinational Matrix Statistical Metrics

The code below uses the data in each column of the combinational matrix to compute their bandgap means and standard deviations in eV. View the updated combinational matrix afterwards to see the results

In [26]:
combinational_means: list[float]
combinational_stds: list[float]
combinational_means, combinational_stds = createStatisticalMetrics(combinational_matrix, bandgap_bins)
combinational_matrix.loc["total"] = list(combinational_matrix.sum())              # type: ignore
combinational_matrix.loc["mean"] = list(combinational_means)                      # type: ignore
combinational_matrix.loc["std"] = list(combinational_stds)                        # type: ignore
combinational_matrix

,PrimaryAmine,SecondaryAmine,TertiaryAmine,PrimaryAmine SecondaryAmine,total
0.0,0.000000,0.000000,0.000000,0.000000,0.000000
0.5,0.000000,0.000000,0.000000,0.000000,0.000000
1.0,1.000000,2.000000,2.000000,0.000000,5.000000
1.5,4.000000,5.000000,3.000000,1.000000,13.000000
2.0,5.000000,8.000000,2.000000,0.000000,15.000000
2.5,8.000000,6.000000,4.000000,0.000000,18.000000
3.0,11.000000,6.000000,3.000000,3.000000,23.000000
3.5,10.000000,1.000000,1.000000,0.000000,12.000000
4.0,2.000000,3.000000,1.000000,0.000000,6.000000
4.5,2.000000,1.000000,0.000000,0.000000,3.000000


## Combinational Matrix To Excel Export

Table to Excel Sheet Conversion

The code below creates an excel sheet for the combinational matrix excel sheet and enters in the <b>pandas</b> dataframe into the sheet. The sheet name is assigned and the columns are width assigned to their name for spacing and visual apperance. 

In [27]:
combinational_sheet_name: str = "combinational_matrix"
combinational_matrix.to_excel(writer, sheet_name=combinational_sheet_name, freeze_panes=(1, 1))
combinational_sheet = writer.sheets[combinational_sheet_name]
combinational_sheet.set_column(0, 0, 7)
combinational_sheet_columns: list[str] = [str(col) for col in combinational_matrix.columns][1:]
for j, col in enumerate(combinational_sheet_columns):
    combinational_sheet.set_column(j+1, j+1, len(col)+7)


Stacked Percentage Chart Creation

Below is code using the [xlsxwriter chart interface](https://xlsxwriter.readthedocs.io/chart.html) that converts the independent rows and columns of the combinational matrix into a stacked percentage chart which shows the percent distribution of each functional group combination set in each bandgap category. The process creates the chart and references the combinational matrix inside of the excel sheet with the <b>writer</b> object to assign the data into stacked percentage categories shown in the graph. Labeling of bar chart categories with the columns, unique colors per cateogry, category percentages, and chart dimensions are all assigned during the process. Please reference the [xlsxwriter chart interface](https://xlsxwriter.readthedocs.io/chart.html) for details on the excel coding interface style below.

In [28]:
##### Combinational Matrix Chart Setup #####
stacked_chart = writer.book.add_chart({'type': 'column', 'subtype': 'stacked'})             # type: ignore
combinational_rows = combinational_matrix.loc[combinational_matrix["total"] > 0]
combinational_bandgap_bins = [index for index in combinational_rows.index if index not in ["total", "mean", "std"]]
fg_groups = [group for group in combinational_matrix.columns if group != "total"]

##### Bandgap Bin Sorted Combinational Functional Group Percentage Label Creation #####
custom_labels = {fg: [] for fg in fg_groups}
for index, item in combinational_rows.iterrows():

    ##### Skip Non-Functional Group Categories #####
    if index in ["total", "mean", "std"]: continue

    ##### Bandgap Categorized Combinational Group Percentages Calculation And Labeling Setup #####
    row: list[float] = list(item)
    total: float = row[-1]
    counts: list[float] = row[:-1]
    percentages: list[str] = [str(round((count/total)*100, 2)) + "%" if count else " "  for count in counts]

    ##### combinational Groups Labeling To Chart Setup #####
    for j, p in enumerate(percentages):
        custom_labels[fg_groups[j]].append({'value': p, 'font': {'color': COLORS[j].label_color}})

##### Stacked Chart Plot Insertion #####
bandgap_row_lookup: dict[float, int] = {bg_value: 1+row for row, bg_value in enumerate(bandgap_bins)}
start_row = bandgap_row_lookup[combinational_bandgap_bins[0]]
for j, (name, item) in enumerate(combinational_rows.items()):
    if name == "total": continue
    stacked_chart.add_series({
        'values':       [combinational_sheet_name, start_row, j+1, start_row+len(combinational_bandgap_bins)-1, j+1], 
        'categories':   [combinational_sheet_name, start_row, 0, start_row+len(combinational_bandgap_bins)-1, 0],
        'data_labels':  {'value': True, 'custom': custom_labels[str(name)]},
        'fill':         {'color': COLORS[j].bar_color},
        'name':         str(name),
    })

##### Stacked Chart Axis Labeling #####
stacked_chart.set_x_axis({
    'name': 'Optical Band Gaps (eV)',
    'name_font': {'size': 30, 'bold': True},
    'num_font':  {'italic': True },
})

stacked_chart.set_y_axis({
    'name': 'Structure Count',
    'name_font': {'size': 30, 'bold': True},
    'num_font':  {'italic': True },
})


##### Stacked Chart Size Formatting #####
stacked_chart.set_size({
    'width': 1860,
    'height': 720
})

##### Stacked Chart Title #####
stacked_chart.set_title({
    'name': combinational_sheet_name
})

##### Stacked Chart Legend #####
stacked_chart.set_legend({'font': {'size': 13, 'bold': True}})

##### Stacked Chart Sheet Insertion #####
combinational_sheet.insert_chart("A" + str(len(combinational_matrix.index) + 5), stacked_chart)


0

Close the excel sheet below to finish excel exporting

In [29]:
writer.close()